In [1]:
# Import dependencies

import pandas as pd
import numpy as np
import datetime as dt
import scipy.stats as sp
import statsmodels.api as sm
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.core.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Upload your csv file with gaps using the File interface on the left.
# Then enter name of CSV file you uploaded and execute this cell.
file_name = "base_file.csv"

In [4]:
# Read the CSV file into a dataframe and plot the time series.

grace_df = pd.read_csv(file_name, index_col=0)
grace_df.index = pd.to_datetime(grace_df.index)
grace_df.index = grace_df.index.to_series().dt.strftime("%Y-%m")
grace_df.index = pd.to_datetime(grace_df.index)

total_series = go.Scatter(name='Groundwater Storage (Calculated)', x=grace_df.index, y=grace_df.iloc[:, 0].values)
plot_series = [total_series]
layout = go.Layout(title="Groundwater Storage (Calculated)", xaxis=dict(title='Date',), yaxis=dict(title='Groundwater Storage (Calculated)',),)
chart_obj = go.Figure(data=plot_series, layout=layout)
chart_obj.show()

In [5]:
# Process trends and seasonality prior to 2017 break

grace_df_subset_1 = grace_df.loc[grace_df.index <= pd.to_datetime('2017-06')]
grace_df_subset_1.index = pd.to_datetime(grace_df_subset_1.index)

grace_df_res_1 = sm.tsa.seasonal_decompose(grace_df_subset_1, model='additive', freq=12)

grace_df_res_trend_1 = grace_df_res_1.trend.copy()
grace_df_res_seasonal_1 = grace_df_res_1.seasonal.copy()
grace_df_res_resid_1 = grace_df_res_1.resid.copy()

In [6]:
# Process trends and seasonality prior after 2018

grace_df_subset_2 = grace_df.loc[grace_df.index >= pd.to_datetime('2018-06')]
grace_df_subset_2.index = pd.to_datetime(grace_df_subset_2.index)

grace_df_res_2 = sm.tsa.seasonal_decompose(grace_df_subset_2, model='additive', freq=12)

grace_df_res_trend_2 = grace_df_res_2.trend.copy()
grace_df_res_seasonal_2 = grace_df_res_2.seasonal.copy()
grace_df_res_resid_2 = grace_df_res_2.resid.copy()

In [7]:
# Joining the two subsets 

grace_df_subset = grace_df_subset_1.append(grace_df_subset_2)
grace_df_res_trend = grace_df_res_trend_1.append(grace_df_res_trend_2)
grace_df_res_seasonal = grace_df_res_seasonal_1.append(grace_df_res_seasonal_2)
grace_df_res_resid = grace_df_res_resid_1.append(grace_df_res_resid_2)

grace_df_res_trend = grace_df_res_trend.dropna()
grace_df_res_trend_index = grace_df_res_trend.index.to_list()
grace_df_res_trend_index = np.array(grace_df_res_trend_index, dtype=np.datetime64)
grace_df_res_trend_index = np.array(grace_df_res_trend_index, dtype=np.timedelta64)
grace_df_res_trend_index = np.array(grace_df_res_trend_index, dtype=np.long)

grace_df_res_trend_index_dates = []
for date in grace_df_res_trend_index:
    grace_df_res_trend_index_dates.append(dt.datetime.fromtimestamp(date/1000000))

grace_df_res_trend = grace_df_res_trend.to_frame()
grace_df_res_seasonal = grace_df_res_seasonal.to_frame()
grace_df_res_resid = grace_df_res_resid.to_frame()

In [8]:
# Adjust linear regressions and export files of components

slope, intercept, r_value, p_value, std_err = sp.linregress(grace_df_res_trend_index, grace_df_res_trend.iloc[:, 0].values)

dates = pd.date_range(grace_df_subset.index[0], grace_df_subset.index[len(grace_df_subset.index)-1], freq='MS')

df1 = pd.DataFrame(np.nan, index=dates, columns=['Groundwater Storage (Calculated)'])
df1.index.name = 'Date'
grace_df_subset_total = df1.fillna(grace_df_subset)
grace_df_subset_total.rename(columns={'Groundwater Storage (Calculated)':'Orignal GWSA'}, inplace=True)
grace_df_subset_total.to_csv('original.csv')

df2 = pd.DataFrame(np.nan, index=dates, columns=['trend'])
df2.index.name = 'Date'
grace_df_res_trend_total = df2.fillna(grace_df_res_trend)
grace_df_res_trend_total.rename(columns={'trend':'Trend'}, inplace=True)
grace_df_res_trend_total.to_csv('trend.csv')

df3 = pd.DataFrame(np.nan, index=dates, columns=['seasonal'])
df3.index.name = 'Date'
grace_df_res_seasonal_total = df3.fillna(grace_df_res_seasonal)
grace_df_res_seasonal_total.rename(columns={'seasonal':'Seasonality'}, inplace=True)
grace_df_res_seasonal_total.to_csv('seasonal.csv')

df4 = pd.DataFrame(np.nan, index=dates, columns=['resid'])
df4.index.name = 'Date'
grace_df_res_resid_total = df4.fillna(grace_df_res_resid)
grace_df_res_resid_total.rename(columns={'resid':'Resid'}, inplace=True)
grace_df_res_resid_total.to_csv('resid.csv')


In [9]:
# Plotting the data

fig = make_subplots(rows=4, cols=1, subplot_titles=("Orignal GWSA", "Trend", "Seasonality", "Resid"))

fig.add_trace(go.Scatter(
                x=grace_df_subset_total.index,
                y=grace_df_subset_total.iloc[:, 0].values,
                name='Groundwater Storage (Calculated)',),
              row=1, col=1)

fig.add_trace(go.Scatter(
    x=grace_df_res_trend_index_dates,
    y=grace_df_res_trend.iloc[:, 0].values,
    mode='markers',
    name='',
    line=dict(color='blue'),),
    row=2, col=1)

fig.add_trace(go.Scatter(
                x=grace_df_res_trend_total.index,
                y=grace_df_res_trend_total.iloc[:, 0].values,
                name='Trend',),
              row=2, col=1)

fig.add_trace(go.Scatter(
    x=[dt.datetime.fromtimestamp(grace_df_res_trend_index[0]/1000000), dt.datetime.fromtimestamp(grace_df_res_trend_index[-1]/1000000)],
    y=[slope * grace_df_res_trend_index[0] + intercept, slope * grace_df_res_trend_index[-1] + intercept],
    mode='lines',
    name='{0}x + {1}'.format(str(round(slope, 2)), str(round(intercept, 2))),
    line=dict(color='black', dash='dash'),),
    row=2, col=1)

fig.add_trace(go.Scatter(
                x=grace_df_res_seasonal_total.index,
                y=grace_df_res_seasonal_total.iloc[:, 0].values,
                name='Seasonality',),
              row=3, col=1)

fig.add_trace(go.Scatter(
                x=grace_df_res_resid_total.index,
                y=grace_df_res_resid_total.iloc[:, 0].values,
                name='Resid',),
              row=4, col=1)

fig.update_layout(showlegend=False)

chart_obj = go.Figure(fig)

chart_obj.show()

print('{0}x + {1}'.format(str(slope), str(intercept)))

3.751646726889165e-14x + -42.410819200004056


In [10]:
# Combine the components to fill in missing data

grace_df_res_seasonal_total.rename(columns={'Seasonality':'Orignal GWSA'}, inplace=True)
grace_df_res_resid_total.rename(columns={'Resid':'Orignal GWSA'}, inplace=True)

seasonal_df = grace_df_res_seasonal_total + grace_df_res_resid_total
seasonal_df.dropna(inplace=True)

seasonal_mean_df = seasonal_df.groupby(seasonal_df.index.strftime("%m"))
seasonal_mean_df = seasonal_mean_df.mean()

indexes = list(range(0, len(grace_df_subset_total.index)))

grace_df_subset_completed = grace_df_subset_total.copy()

for date, i in zip(dates, indexes):
  if np.isnan(grace_df_subset_total.loc[grace_df_subset_total.index == date].values[0][0]):
    mes = grace_df_subset_total.index[i].month
    if mes < 10:
      mes = '0'+str(mes)
    else:
      mes = str(mes)
    date_num = np.datetime64(date.to_pydatetime()).astype(np.long)

    grace_df_subset_completed.loc[grace_df_subset_completed.index == date] = (seasonal_mean_df[seasonal_mean_df.index == mes].values[0][0]) + (slope * date_num + intercept) 

total_series = go.Scatter(name='Orignal GWSA', x=grace_df_subset_total.index, y=grace_df_subset_total.iloc[:, 0].values)
completed_series = go.Scatter(name='Orignal GWSA (Completed)', x=grace_df_subset_completed.index, y=grace_df_subset_completed.iloc[:, 0].values)



In [11]:
# Plot final results 

plot_series = [completed_series, total_series]
layout = go.Layout(title="Orignal GWSA (Calculated)", xaxis=dict(title='Date',), yaxis=dict(title='Groundwater Storage',),)
chart_obj = go.Figure(data=plot_series, layout=layout)
chart_obj.show()


In [12]:
# Export file

grace_df_subset_completed.to_csv("completed.csv")

grace_df_res_seasonal_total.rename(columns={'Orignal GWSA':'Seasonality'}, inplace=True)
grace_df_res_resid_total.rename(columns={'Orignal GWSA':'Resid'}, inplace=True)